In [4]:
%load_ext autoreload
%autoreload 2

from helpers import *

from surprise import Dataset
from surprise import Reader
from ALS_implementation import ALS
from surprise.model_selection import GridSearchCV

path_dataset = "data/data_train.csv"
data = load_data(path_dataset)
dataset = Dataset.load_from_df(data[['user', 'movie', 'rating']], Reader(rating_scale=(1, 5)))

data.head() 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


,user,movie,rating
0,r44,c1,4
1,r61,c1,3
2,r67,c1,4
3,r72,c1,3
4,r86,c1,5


![alt text](ALS.png "Title")

In [ ]:
param_grid = {"n_epochs": [25], "num_features": [8,9], "lambda_all": [0.08,0.081,0.082]}
gs = GridSearchCV(ALS, param_grid, measures=['rmse', 'mae'], cv=2, n_jobs=-1, joblib_verbose=1000) 
gs.fit(dataset)
print(gs.best_score['rmse'])
print(gs.best_params['rmse'])
print(gs.cv_results)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


C:\Users\Adrien G\Anaconda3\lib\site-packages\joblib\externals\loky\process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


In [2]:
param_grid = {"n_epochs": [30], "num_features": [8,9], "lambda_all": [0.08,0.081,0.082]}
gs = GridSearchCV(ALS, param_grid, measures=['rmse', 'mae'], cv=2, n_jobs=-1, joblib_verbose=1000) 
gs.fit(dataset)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed: 10.7min
[Parallel(n_jobs=-1)]: Done   2 out of  12 | elapsed: 11.3min remaining: 56.5min
[Parallel(n_jobs=-1)]: Done   3 out of  12 | elapsed: 11.7min remaining: 35.0min
[Parallel(n_jobs=-1)]: Done   4 out of  12 | elapsed: 11.9min remaining: 23.8min
[Parallel(n_jobs=-1)]: Done   5 out of  12 | elapsed: 12.2min remaining: 17.0min
[Parallel(n_jobs=-1)]: Done   6 out of  12 | elapsed: 12.5min remaining: 12.5min
[Parallel(n_jobs=-1)]: Done   7 out of  12 | elapsed: 12.7min remaining:  9.1min
[Parallel(n_jobs=-1)]: Done   8 out of  12 | elapsed: 13.0min remaining:  6.5min
[Parallel(n_jobs=-1)]: Done   9 out of  12 | elapsed: 13.3min remaining:  4.4min
[Parallel(n_jobs=-1)]: Done  10 out of  12 | elapsed: 13.4min remaining:  2.7min
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed: 13.9min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapse

In [3]:
print(gs.best_score['rmse'])
print(gs.best_params['rmse'])
print(gs.cv_results)

1.0072962415827464
{'n_epochs': 30, 'num_features': 8, 'lambda_all': 0.082}
{'split0_test_rmse': array([1.00746965, 1.00712822, 1.00679865, 1.01038207, 1.00993078,
       1.00949457]), 'split1_test_rmse': array([1.00848898, 1.00813552, 1.00779383, 1.00995478, 1.00953154,
       1.00912174]), 'mean_test_rmse': array([1.00797932, 1.00763187, 1.00729624, 1.01016842, 1.00973116,
       1.00930815]), 'std_test_rmse': array([0.00050966, 0.00050365, 0.00049759, 0.00021365, 0.00019962,
       0.00018642]), 'rank_test_rmse': array([3, 2, 1, 6, 5, 4], dtype=int64), 'split0_test_mae': array([0.81234003, 0.81229345, 0.81225508, 0.81448789, 0.81436258,
       0.81424671]), 'split1_test_mae': array([0.81283652, 0.8127803 , 0.81273164, 0.81353252, 0.81343342,
       0.81334317]), 'mean_test_mae': array([0.81258828, 0.81253688, 0.81249336, 0.8140102 , 0.813898  ,
       0.81379494]), 'std_test_mae': array([0.00024825, 0.00024343, 0.00023828, 0.00047769, 0.00046458,
       0.00045177]), 'rank_test_mae'

In [6]:
submission = pd.read_csv("data/sample_submission.csv")
trainset = dataset.build_full_trainset()
gsbf = gs.best_estimator['rmse'].fit(trainset)

number of items in preprocess_trainset_data: 1000, number of users: 10000

start the ALS algorithm...
RMSE on training set: 1.0279557692829417.
RMSE on training set: 1.002643700880776.
RMSE on training set: 0.984104400950518.
RMSE on training set: 0.9725554764695716.
RMSE on training set: 0.9649053241359405.
RMSE on training set: 0.9593962705374679.
RMSE on training set: 0.9554039026823504.
RMSE on training set: 0.952536228342509.
RMSE on training set: 0.9504453053662352.
RMSE on training set: 0.948866722777238.
RMSE on training set: 0.9476306025435011.
RMSE on training set: 0.946633145835352.
RMSE on training set: 0.9458099253295328.
RMSE on training set: 0.945119423907407.
RMSE on training set: 0.9445336891663806.
RMSE on training set: 0.9440329854033788.
RMSE on training set: 0.9436026810899607.
RMSE on training set: 0.9432314284306135.
RMSE on training set: 0.9429101013482459.
RMSE on training set: 0.9426311748098132.
RMSE on training set: 0.9423883539770682.
RMSE on training set: 

In [7]:
def predict(user, movie):
    return int(round(recommenders[cluster_dict[movie]].predict(user, movie).est))

In [8]:
submission['Prediction'] = [int(round(gsbf.predict(user, movie).est)) for [user, movie] in submission['Id'].str.split('_')]
submission.to_csv(r'data/submission_als_adri_1.csv', index=False)